# Changer :
Définition de saut,
Ruines 1-1 numba

In [ ]:
import numpy as np
import math
import matplotlib.pyplot

## Q1

Dans cette première modélisation simplifiée, on considère donc que le prix $P_t$ est un processus de Poisson de paramètres $\lambda, \nu$ où $\nu$ est la loi des incréments $J_n$. 

Pour un temps d'attente moyen entre deux sauts de $300s$, on prend $\lambda = \dfrac{1}{300}$


### Q1 -1 Probabilité de ruine


In [ ]:
# On identifie le processus par le processus de Poisson composé, qui finit à un temps fixé T
# On crée des fonctions pour modéliser le processus.

#### On définit nu, la loi des incréments
# Ancien : plus lent
saut_1_ancien = lambda x: np.random.choice([-1, 1], size=x, replace=True, p=[0.5, 0.5]) #correspond à m=1
saut_2_ancien = lambda x: np.random.choice([-3, -2, -1, 1, 2, 3], size=x, replace=True, p=0.5*np.array([1/6, 1/3, 1/2, 1/2, 1/3, 1/6])) #correspond à m=3

# Nouveau : beaucoup plus vite
value_1 = np.array([-1, 1])
value_2 = np.array([-3, -2, -2, -1, -1, -1, 1, 1, 1, 2, 2, 3])
saut_1 = lambda x : value_1[np.random.randint(low=2, size=x)]
saut_2 = lambda x : value_2[np.random.randint(low=12, size=x)]

# Les paramètres
P0 = 35
T = 4*60*60
lamb = 1/300

In [ ]:
## Echantillionage d'importance

from numba import jit

@jit(nopython=True)# Function is compiled to machine code when called the first time
def inf_echantillon_importance(N, J, P0, lambT, s, f_dic): 
  ruines = 0
  for i in range(len(N) - 1):
    somme = P0
    ruine = 0.
    for j in range(N[i], N[i + 1]):
      somme += J[j]
      if somme < 0:
        # Calculer L_T
        X_T_f = np.sum(f_dic[ J[N[i]:N[i+1]] ])
        L_T = np.exp(X_T_f - (s - 1) * lambT)
        ruine = 1 / L_T
        break
    ruines += ruine
  return ruines

def trajectoire_importance(P0, T, lamb, m, size, f):
  if m == 1:
    value = np.array([-1, 1])
    p = np.array([1/2, 1/2])
  else:
    value = np.array([-3, -2, -1, 1, 2, 3])
    p = np.array([1 / 12, 1 / 6, 1 / 4, 1 / 4, 1 / 6, 1 / 12])
    
  # Nouvelle loi
  s = np.sum(np.exp(f(value)) * p)
  new_lamb = lamb * s
  new_p = np.exp(f(value)) * p / s
  
  # f_dic : i -> f(i) sous la forme dictionnaire
  f_dic = np.zeros(10)
  for i in value:
    f_dic[i] = f(i)
  print(s)
  print(new_lamb * T * np.sum(value * new_p)) # Esperance (On veut que ce soit environ 0)
  #if abs(new_lamb * T * np.sum(value * new_p)) > 100:
  #  return 0
  N = np.random.poisson(lam=new_lamb * T, size=size + 1)
  N[0] = 0
  N = N.cumsum()                    # La valeur N[i] - N[i - 1] est égale à Ni pour le i-ième échantillon
                                    # Donc la somme des sauts entre indice N[i] et N[i + 1] - 1 suit la loi voulue
  
  J = np.random.choice(value, size=N[-1] + 1, p=new_p)
  res = inf_echantillon_importance(N, J, P0, lamb * T, s, f_dic)
  proba = res / size
  return proba

In [10]:
%%time
m = 3
M = int(1e6)
f = lambda x : 0.5

p_r = trajectoire_importance(P0, T, lamb, 3, M, f)
R_IC = 1.96*np.sqrt(p_r*(1-p_r))/np.sqrt(M) #rayon de l'intervalle de confiance
print(p_r)
print(R_IC)

1.6487212707001282
4.3930759582092675e-15
0.005009256970206878
0.00013837328366338213
CPU times: user 1.77 s, sys: 389 ms, total: 2.16 s
Wall time: 2.16 s


In [6]:
## Monte-Carlo Naif avec numba

from numba import jit

@jit(nopython=True)
def inf_echantillon(N, J, P0): # Function is compiled to machine code when called the first time
  ruines = 0
  for i in range(len(N) - 1):
    somme = P0
    ruine = 0
    for j in range(N[i], N[i + 1]):
      somme += J[j]
      if somme < 0:
        ruine = 1
        break
    ruines += ruine
  return ruines

def trajectoire(P0, T, lamb, saut, size):
  if size > int(1e7):
    sizes = size
    size = int(1e7)
    proba = 0
    for i in range(sizes // size):
      N = np.random.poisson(lam=lamb * T, size=size + 1)
      N[0] = 0
      N = N.cumsum()                    # La valeur N[i] - N[i - 1] est égale à Ni pour le i-ième échantillon
                                        # Donc la somme des sauts entre indice N[i] et N[i + 1] - 1 suit la loi voulue
      J = saut(N[-1] + 1)
      res = inf_echantillon(N, J, P0)
      proba += res / size / (sizes // size)
    return proba
  else:
    N = np.random.poisson(lam=lamb * T, size=size + 1)
    N[0] = 0
    N = N.cumsum()                    # La valeur N[i] - N[i - 1] est égale à Ni pour le i-ième échantillon
                                      # Donc la somme des sauts entre indice N[i] et N[i + 1] - 1 suit la loi voulue
    J = saut(N[-1] + 1)
    res = inf_echantillon(N, J, P0)
    proba = res / size
    return proba

In [7]:
%%time
M = int(1e6)
p_r = trajectoire(P0, T, lamb, saut_2, size=M)
R_IC = 1.96*np.sqrt(p_r*(1-p_r))/np.sqrt(M) #rayon de l'intervalle de confiance
print(p_r)
print(R_IC)

#print(trajectoire(P0, T, lamb, saut_1, size=int(1e7)))

#m=1
#0.00279 pour P0 = 20 M = 10^5 (total time 161 ms)
#5e-07 pour P0=35 et M=10^7 total time : 16.1 s
#quand on fait la moyenne sur 10 essais à M=10^7, on trouve proba_emp = 2.009975 e-7 et sigma_emp = 3.6 e-7 (tout ça pour m=1)

#m=3
#proba_emp=0.0043245, sigma_emp = 7.1273066441679e-05

0.004342
0.00012887125999809886
CPU times: user 593 ms, sys: 117 ms, total: 710 ms
Wall time: 711 ms


In [ ]:
## Splitting et MCMC - Méthode 3



def NiveauxSplitting(a,seuil,n,lamb,T,p,P0,saut):

    """
    Fonction qui renvoie une estimation des niveaux
    de splitting a_1, a_2, ..., a_k tels que P(Phi_T <= a_k | Phi_T <= a_{k-1}) = 0.1 = seuil
    (où Phi_T : inf de P_t pour t dans [0;T])
    Ces niveaux sont des quantiles d'une loi conditionnelle.
    On utilise l'inversion de la fonction de repartition empirique de 
    cette loi afin d'estimer un quantile par
    le quantile empirique.
    On a a = a_k < a_{k-1} < ... < a_0= + infini (dans notre problème, a = 0)
    La fonction renvoie quantiles = [a_1, ..., a_k]
    """
    ## Estimation du premier niveau a_1: c'est le 
    ## quantile d'une loi non conditionnelle.
    ## On l'estime ici par la methode ergodique

    liste_Phi = np.zeros(n)

    liste_sauts = liste_sts(lamb,T,saut)

    for l in range(n):
        coloriage = liste_sauts[:,np.random.binomial(1,p,size = len(liste_sauts[0])) ==1]

        liste_sauts_tilde = liste_sts((1-p)*lamb,T,saut)
        new_liste_sauts = np.concatenate((coloriage,liste_sauts_tilde),axis=1)
        new_liste_sauts = tri_temps(new_liste_sauts)

        liste_sauts = new_liste_sauts
        liste_Phi[l] = Phi(liste_sauts,P0)

    liste_Phi.sort()

    quantiles = np.array([liste_Phi[int(np.ceil(seuil*n))-1]])    

    while quantiles[-1] > a:
        print("Inside while")
        liste_Phi = np.zeros(n)

        
        while Phi(liste_sauts,P0)>=quantiles[-1]:
            liste_sauts = liste_sts(lamb,T,saut)
        ## Simulation du processus AR(1) conditionnel
    
        for l in range(n):
            coloriage = liste_sauts[:,np.random.binomial(1,p,size = len(liste_sauts[0])) ==1]

            liste_sauts_tilde = liste_sts((1-p)*lamb,T,saut)
            new_liste_sauts = np.concatenate((coloriage,liste_sauts_tilde),axis=1)
            new_liste_sauts = tri_temps(new_liste_sauts)

            if Phi(new_liste_sauts,P0)<quantiles[-1]:
                liste_sauts = new_liste_sauts

            liste_Phi[l] = Phi(liste_sauts,P0)
    
        liste_Phi.sort()
        quantiles = np.append(quantiles, liste_Phi[int(np.ceil(seuil*n))-1] )

    
    ## On selectionne les niveaux a_{k-1},..., a_1 strictement au dessus de a
    quantiles = quantiles[:-1]
    ## On rajoute a
    quantiles = np.append(quantiles,a)

    return quantiles


def Phi(liste_sauts,P0):#fonction qui renvoie l'inf des valeurs de X aux instants de saut
    #ie l'inf de P0+cumsum(incréments) 
    if len(liste_sauts[0]) == 0:
        return P0
    liste_prix = P0+np.cumsum(liste_sauts[1,:])
    prix_min = np.min(liste_prix)
    return prix_min

def liste_sts(lbda,T,saut):
    
    N = np.random.poisson(lbda*T)
    liste_temps_sauts = np.random.uniform(low = 0, high = T, size = N)
    liste_temps_sauts_triee = [np.sort(liste_temps_sauts)]
    liste_increments = [saut(N)]
    #renvoie un array de N colonnes et 2 lignes: 1ere ligne pour les temps des sauts (T_n), deuxième ligne pour leurs amplitudes (J_n)
    return np.concatenate((liste_temps_sauts_triee,liste_increments),axis=0)

def tri_temps(new_liste_sauts):
    ordre = [new_liste_sauts[0,:].argsort()]
    liste_sauts_triee = np.take_along_axis(new_liste_sauts, np.concatenate((ordre,ordre),axis=0), axis=1) 
    return liste_sauts_triee


def MCMC(M,p,lamb,liste_a,P0,saut): 

    liste_pi = np.zeros(len(liste_a)) #estimateurs des probabilités conditionnelles

    liste_indicatrices = np.zeros(M,dtype=bool) 
    """
    le k-ieme élém. de liste_indicatrices vaut True si le prix devient négatif avant l'instant T lors du k-ieme essai; False sinon
    """

    #Loi non conditionnelle
    
    liste_sauts = liste_sts(lamb,T,saut) 


    for l in range(M):
        coloriage = liste_sauts[:,np.random.binomial(1,p,size = len(liste_sauts[0])) ==1]

        liste_sauts_tilde = liste_sts((1-p)*lamb,T,saut)
        new_liste_sauts = np.concatenate((coloriage,liste_sauts_tilde),axis=1)
        new_liste_sauts = tri_temps(new_liste_sauts)

        liste_sauts = new_liste_sauts
        liste_indicatrices[l] = (Phi(liste_sauts,P0)<liste_a[0])

    liste_pi[0] = np.mean(liste_indicatrices)
    
        


    for k in range(1,len(liste_a)):
        liste_indicatrices = np.zeros(M,dtype=bool)  

        #Initialisation 

        while Phi(liste_sauts,P0)>=liste_a[k-1]:
          
            liste_sauts = liste_sts(lamb,T,saut)

      
        for l in range(M):
            coloriage = liste_sauts[:,np.random.binomial(1,p,size = len(liste_sauts[0])) ==1]

            liste_sauts_tilde = liste_sts((1-p)*lamb,T,saut)
            new_liste_sauts = np.concatenate((coloriage,liste_sauts_tilde),axis=1)
            new_liste_sauts = tri_temps(new_liste_sauts)

            if Phi(new_liste_sauts,P0)< liste_a[k-1]:
                liste_sauts = new_liste_sauts
            liste_indicatrices[l] = (Phi(liste_sauts,P0) < liste_a[k])


        liste_pi[k] = np.mean(liste_indicatrices)
    

    proba_prix_negatif = np.prod(liste_pi)
        
    return proba_prix_negatif

In [ ]:
P0 = 35 
T = 4*60*60 #conversion en secondes
lamb =  1/300 
M = int(1e4) 

a=0


#Choix des paramètres pour les niveaux de splitting et la simulation par chaîne de Markov
seuil = 0.03
p=0.5

liste_a = NiveauxSplitting(a,seuil,M,lamb,T,p,P0,saut_1)
print(liste_a)
#Donne [22. 14.  8.  3.  0.] pour m=1 et seuil = 0.05
#Donne [20.  9.  1.  0.] pour m=3 et seuil = 0.2

In [ ]:
probas = np.zeros(10)

M = int(1e5)
n=10

for i in range(n):
    probas[i] = MCMC(M, p,lamb,liste_a,P0,saut_1)

proba_emp = np.mean(probas)
sigma_emp = np.std(probas)/np.sqrt(n) #écart-type empirique de la moyenne des n estimateurs
print(proba_emp)
print(sigma_emp)
#Réponse pour m=1 ie k=0 et P0=35 : proba = 3.333674930684664e-07, variance = 5 10^-8
#m=3 et P0=35: proba = 0.004309577501008395, variance = 0.00010259619121779947
#m=1 liste_a = [24. 17. 12.  7.  3.  0.] seuil =0.1 M=10^4 n=10 proba_emp=4.619226215866903e-07 sigma_emp= 9.561801048257746e-08
#m=1 liste_a= [20. 12.  5.  0.] seuil = 0.03 P0=35 M=10^4 n=10 proba_emp = 3.6218700696e-07 sigma_emp = 7.599237232859178e-08 
#same avec M=10^5 proba_emp = 3.4354401671979703e-07. sigma_emp= 1.3033966706232877e-08 LE MEILLEUR QU ON AIT

In [ ]:
print("Estimation de la probabilité de ruine pour m=1 par méthode de Splitting/MCMC: {:09.8f}+/-{:09.8f}".format(proba_emp,sigma_emp))